# Understand python code agent using OpenAI

In [27]:
import dotenv

dotenv.load_dotenv()

True

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [29]:
from langchain.agents import tool
from langchain_community.document_loaders.generic import GenericLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language
from langchain_openai import OpenAIEmbeddings

@tool
def get_parsed_python_file(path: str) -> int:
    """Python Parser Tool: Returns the content of the python code file"""
    parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
    loader = GenericLoader.from_filesystem(path, glob="**/*", suffixes=[".py"], parser=parser)
    documents = loader.load()
    return str(documents[0].page_content)

get_parsed_python_file.invoke("../docs/sample.py")

tools = [get_parsed_python_file]

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful code documenting assistant, but don't know the user's source code",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [31]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [32]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [33]:
# list(agent_executor.stream({"input": "Write a docstring for the source code defined in the file ../docs/sample.py"}))

In [34]:
# list(agent_executor.stream({"input": "Write a docs for the file ../docs/sample.py"}))

# Understand Python using LLama2

In [35]:
# from langchain.llms import CTransformers
# from llama_cpp import Llama
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# llm = CTransformers(model='../models/llama-2-7b-chat.ggmlv3.q8_0.bin', model_type='llama', config={'max_new_tokens': 1024})
# llm = Llama(model_path="../models/Nous-Hermes-Llama2-13b-q8_0.gguf", n_ctx=4096)

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="../models/nous-hermes-llama-2-7b.q4_0.gguf",
    temperature=0,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    n_ctx=4096,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 18 key-value pairs and 291 tensors from ../models/nous-hermes-llama-2-7b.q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = nous-hermes-llama-2-7b.ggmlv3.q4_0.bin
llama_model_loader: - kv   2:                        general.description str              = converted from legacy GGJTv3 format
llama_model_loader: - kv   3:                       llama.context_length u32              = 2048
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 11008
llama_model_loa

In [50]:
import re
from langchain.prompts import StringPromptTemplate, PromptTemplate
from typing import List

template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following questions as best you can. You have access to the following tools:

get_parsed_python_file: A tool to load python files from the local source code. The input is the path to the python file

Strictly use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python Parser Tool]
Action Input: the input to the action, should be the file needed to generate documentation for.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For examples:
Question: Write me a documentation of the python file ../test.py
Thought: First, I need to read the python file ../test.py
Action: get_parsed_python_file
Action Input: ../test.py
Observation: def addNumbers(a, b):
                 sum = a + b
                 return sum
Thought: I should generate documentation of this code.
Action: Generate the documentation of the code
Action Input: def addNumbers(a, b):
                 sum = a + b
                 return sum
Observation: This file contains two functions:

            ### addNumbers(a, b)

            This function takes two parameters `a` and `b` and returns their sum.

            #### Parameters

            - `a` (int): The first number.
Thought: I now know the final answer.
Final Answer: This file contains two functions:

            ### addNumbers(a, b)

            This function takes two parameters `a` and `b` and returns their sum.

            #### Parameters

            - `a` (int): The first number.

### Input:
{input}

### Response:
{agent_scratchpad}"""

temp_Ins = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Question: {thought}
Query: {query}
Observation: {observation}

### Input:
Make a short summary of useful information from the result observation that is related to the question.

### Response:"""

prompt_Ins = PromptTemplate(
    input_variables=["thought", "query", "observation"],
    template=temp_Ins,
)


class CustomPromptTemplate(StringPromptTemplate):
    """Schema to represent a prompt for an LLM.

    Example:
        .. code-block:: python

            from langchain import PromptTemplate
            prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}")
    """

    input_variables: List[str]
    """A list of the names of the variables the prompt template expects."""

    template: str
    """The prompt template."""

    template_format: str = "f-string"
    """The format of the prompt template. Options are: 'f-string', 'jinja2'."""

    validate_template: bool = False
    """Whether or not to try validating the template."""
    

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        if len(intermediate_steps) > 0:
            regex = r"Thought\s*\d*\s*:(.*?)\nAction\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
            text_match = intermediate_steps[-1][0].log
            if len(intermediate_steps) > 1:
                text_match = 'Thought: ' + text_match


            # print(intermediate_steps)
            
            
            match = re.search(regex, text_match, re.DOTALL)            
            my_list = list(intermediate_steps[-1])

            # print("Observation")
            # print(my_list[1])

            p_INS_temp = prompt_Ins.format(thought=match.group(1).strip(), query=match.group(3).strip(), observation=my_list[1])
            my_list[1] = llm(p_INS_temp)
            my_tuple = tuple(my_list)            
            intermediate_steps[-1] = my_tuple
            
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f" {observation}\nThought:"
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        return self.template.format(**kwargs)
    
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps"],
                              template=template,validate_template=False)

In [52]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from typing import Union, Any

class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
output_parser = CustomOutputParser()

In [53]:
from langchain import LLMChain
from langchain.agents import LLMSingleActionAgent

llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:", "\nObservations:"], 
    allowed_tools=tools
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [54]:
agent_executor.run("Write docs for the file ../docs/sample.py")



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit


 Thought: First, I need to read the python file ../docs/sample.py
Action: get_parsed_python_file
Action Input: ../docs/sample.py


llama_print_timings:        load time =     973.64 ms
llama_print_timings:      sample time =       3.81 ms /    44 runs   (    0.09 ms per token, 11545.53 tokens per second)
llama_print_timings: prompt eval time =   20219.57 ms /   397 tokens (   50.93 ms per token,    19.63 tokens per second)
llama_print_timings:        eval time =    3321.98 ms /    43 runs   (   77.26 ms per token,    12.94 tokens per second)
llama_print_timings:       total time =   23663.62 ms /   440 tokens
/Users/prithvirajchaudhuri/Desktop/Other/Projects/jaac/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit


 Thought: First, I need to read the python file ../docs/sample.py
Action: get_parsed_python_file
Action Input: ../docs/sample.py

Observation:import os

def addNumbers(a, b):
    sum = a + b
    return sum

def printEnv():
    print(os.getenv("TEST"))


...


llama_print_timings:        load time =     973.64 ms
llama_print_timings:      sample time =       0.13 ms /     1 runs   (    0.13 ms per token,  7936.51 tokens per second)
llama_print_timings: prompt eval time =    4863.15 ms /    94 tokens (   51.74 ms per token,    19.33 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    4876.01 ms /    95 tokens
Llama.generate: prefix-match hit


 This file contains one function:

            ### addNumbers(a, b)

            This function takes two parameters `a` and `b` and returns their sum.

            #### Parameters

            - `a` (int): The first number.
Thought: I now know the final answer.
Final Answer: This file contains one function:

            ### addNumbers(a, b)

            This function takes two parameters `a` and `b` and returns their sum.

            #### Parameters

            - `a` (int): The first number.


llama_print_timings:        load time =     973.64 ms
llama_print_timings:      sample time =      11.77 ms /   130 runs   (    0.09 ms per token, 11041.28 tokens per second)
llama_print_timings: prompt eval time =   28840.09 ms /   544 tokens (   53.01 ms per token,    18.86 tokens per second)
llama_print_timings:        eval time =   10543.56 ms /   130 runs   (   81.10 ms per token,    12.33 tokens per second)
llama_print_timings:       total time =   39701.37 ms /   674 tokens


 This file contains one function:

            ### addNumbers(a, b)

            This function takes two parameters `a` and `b` and returns their sum.

            #### Parameters

            - `a` (int): The first number.
Thought: I now know the final answer.
Final Answer: This file contains one function:

            ### addNumbers(a, b)

            This function takes two parameters `a` and `b` and returns their sum.

            #### Parameters

            - `a` (int): The first number.

> Finished chain.


'This file contains one function:\n\n            ### addNumbers(a, b)\n\n            This function takes two parameters `a` and `b` and returns their sum.\n\n            #### Parameters\n\n            - `a` (int): The first number.'